# Finetuning LLaMa 7B for sentiment classification
1. Install dependencies
2. Load model
3. Test prompt
4. Create instruction tuned dataset
5. Run training
6. Test prompt

In [1]:
!pip install transformers accelerate datasets trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-hf",
    device_map="auto",
    torch_dtype=torch.bfloat16
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [4]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = model.generate(**model_inputs,max_new_tokens=50)

In [7]:
decoded = tokenizer.decode(output[0],skip_special_tokens=True)
print(decoded)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:
The movie was rated 4/5 stars.

### Instruction:
Classify the following sentiment into a number.

### Input:
I had a terrible experience.

### Response:
The


In [8]:
from datasets import load_dataset
train_ds, test_ds = load_dataset('imdb', split=['train[:2%]+train[-2%:]', 'test[:2%]+test[-2%:]'])

sentiment_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    inputs       = examples["text"]
    outputs      = examples["label"]
    texts = []
    for input, output in zip(inputs, outputs):
        text = sentiment_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
train_ds = train_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True,)
test_ds = test_ds.shuffle(seed=42).map(formatting_prompts_func, batched = True,)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
print(train_ds)
print(test_ds)
print(train_ds["text"][5])
print(train_ds["label"][:10])

Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Dataset({
    features: ['text', 'label'],
    num_rows: 1000
})
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
Second-feature concerns a young woman in London desperate for a job, happy to accept live-in secretarial position with an elderly woman and her son. Thrillers about people being held in a house against their will always make me a little uneasy--I end up feeling like a prisoner too--but this rather classy B-film is neither lurid nor claustrophobic. It's far-fetched and unlikely, but not uninteresting, and our heroine (Nina Foch) is quick on her feet. Rehashing this in 1986 (as "Dead Of Winter") proved not to be wise, as the plot-elements are not of the modern-day. "Julia Ross" is extremely compact (too short at 65 minutes!) but 

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer,TrainingArguments
import torch
from trl import SFTTrainer

args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    )

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    dataset_text_field="text",
    max_seq_length=256,
    args=args
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 0 has a total capacty of 39.56 GiB of which 10.81 MiB is free. Process 5804 has 39.54 GiB memory in use. Of the allocated memory 38.92 GiB is allocated by PyTorch, and 116.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [8]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Classify the following sentiment into a number.

### Input:
It was a great movie!

### Response:"""
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")
with torch.cuda.amp.autocast():
  output = model.generate(**model_inputs,max_new_tokens=50)

In [ ]:
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)